In [18]:
import pandas as pd
import os
import requests
import urllib3
import fitz


In [ ]:
pdf_path = "informes_respiratorios/Informe-circulacion-virus-respiratorios-SE32-12-08-2025.pdf"

import fitz  

def extract_text_pymupdf(path):
    text_pages = []
    with fitz.open(path) as doc:
        for page in doc:
            text_pages.append(page.get_text("text"))
    return "\n".join(text_pages)

raw_text = extract_text_pymupdf(pdf_path)


print("Caracteres extraídos:", len(raw_text))

Caracteres extraídos: 18993


In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,   # ajusta según longitud de respuesta deseada
    chunk_overlap=150,
    separators=["\n\n", "\n", ". ", " ", ""]
)

docs = text_splitter.create_documents([raw_text], metadatas=[{"source": pdf_path}])
len(docs)

22

In [29]:
def check_ollama(host="http://localhost:11434"):
    try:
        r = requests.get(f"{host}/api/tags", timeout=3)
        return r.ok
    except Exception:
        return False

assert check_ollama(), "Ollama no responde en http://localhost:11434. ¿Está levantado?"

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(model="nomic-embed-text")  # requiere ollama corriendo
vectordb = Chroma.from_documents(
    docs, 
    embedding=embeddings, 
    collection_name="isp_informes",
    persist_directory="chroma_isp"   # opcional: guarda en disco
)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

In [32]:
from langchain_community.llms import Ollama
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

llm = Ollama(model="llama3.1", temperature=0.1)

prompt = ChatPromptTemplate.from_template(
    """Eres un analista de vigilancia epidemiológica. 
Analizas informes semanales del ISP sobre virus respiratorios.

Tu tarea es responder la pregunta exclusivamente en formato JSON válido, sin texto adicional.

Pregunta: {question}

Contexto:
{context}

Responde únicamente con un JSON de la siguiente forma:
[
  {{
    "Hospital": "Nombre del hospital",
    "Fecha": "YYYY-MM-DD",
    "Virus": "Nombre del virus",
    "Casos": <número total de casos>,
    "Casos positivos": <número de casos positivos>,
    "Virus positivos": <número de virus positivos>
  }}
]

Si hay varios hospitales o virus en el contexto, incluye un objeto JSON por cada uno en la lista.
"""
)
def format_docs(docs):
    out = []
    for d in docs:
        out.append(f"[{d.metadata.get('source','')}] {d.page_content[:1500]}")
    return "\n\n".join(out)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

##q1 = "¿Cual fue la cantidad de casos reportados en la SE 32"

print(chain.invoke())

[
  {
    "Hospital": "",
    "Fecha": "2025-12-08",
    "Virus": "Total",
    "Casos": 2247,
    "Casos positivos": 1123.5,
    "Virus positivos": 0.9
  },
  {
    "Hospital": "",
    "Fecha": "2025-12-08",
    "Virus": "VRS",
    "Casos": 922,
    "Casos positivos": 922,
    "Virus positivos": 41
  },
  {
    "Hospital": "",
    "Fecha": "2025-12-08",
    "Virus": "Rinovirus",
    "Casos": 610,
    "Casos positivos": 610,
    "Virus positivos": 27.2
  },
  {
    "Hospital": "",
    "Fecha": "2025-12-08",
    "Virus": "Influenza A",
    "Casos": 156,
    "Casos positivos": 156,
    "Virus positivos": 6.9
  },
  {
    "Hospital": "",
    "Fecha": "2025-12-08",
    "Virus": "Metapneumovirus",
    "Casos": 155,
    "Casos positivos": 155,
    "Virus positivos": 6.9
  },
  {
    "Hospital": "",
    "Fecha": "2025-12-08",
    "Virus": "Parainfluenza",
    "Casos": 134,
    "Casos positivos": 134,
    "Virus positivos": 6
  },
  {
    "Hospital": "",
    "Fecha": "2025-12-08",
    "Virus": 